### Feature Selection with sklearn and Pandas
[link](https://towardsdatascience.com/feature-selection-with-pandas-e3690ad8504b)

[Loading libraries and data](#Loading-libraries-and-data)

[Filter Method](#Filter-Method:)

[Wrapper Method](#Wrapper-Method:)

[Embedded Method](#Embedded-Method:)

[PCA](#PCA)

About the dataset:
We will be using the built-in Boston dataset which can be loaded through sklearn. We will be selecting features using the above listed methods for the regression problem of predicting the “MEDV” column. 

#### Loading libraries and data

In [0]:
#importing libraries
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso

In [0]:
#Loading the dataset
x = load_boston()
df = pd.DataFrame(x.data, columns = x.feature_names)
df["MEDV"] = x.target
X = df.drop("MEDV",1)   #Feature Matrix
y = df["MEDV"]          #Target Variable
df.head()

#### Filter Method:

The filtering here is done using correlation matrix.
1. Plot the Pearson correlation heatmap
2. Select features which has correlation of above 0.5
    The correlation coefficient has values between -1 to 1:
    - A value closer to 0 implies weaker correlation (exact 0 implying no correlation).
    - A value closer to 1 implies stronger positive correlation.
    - A value closer to -1 implies stronger negative correlation.
    
3. Check the correlation of selected features with each other
4. Keep that correlated features which correlation with target variable is higher.
5. Use features for modeling.

In [0]:
#Using Pearson Correlation
plt.figure(figsize=(12,10))
cor = df.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [0]:
#Correlation with output variable
cor_target = abs(cor["MEDV"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.5]
relevant_features

In [0]:
# Check the correlation of selected features with each other
print(df[["LSTAT","PTRATIO"]].corr())
print(df[["RM","LSTAT"]].corr())
# it is seen that the variables RM and LSTAT are highly correlated with each other (-0.613808). 
# Hence we would keep only one variable and drop the other. 
# We will keep LSTAT since its correlation with MEDV is higher than that of RM.
# After dropping RM, we are left with two feature, LSTAT and PTRATIO. 
# These are the final features given by Pearson correlation.

### Wrapper Method:

Feed the features to the selected Machine Learning algorithm and based on the model performance you add/remove the features.

This is an iterative and **computationally expensive** process but it is **more accurate** than the filter method.

#### i. Backward Elimination
1. Feed all the possible features to the model at first.
2. Check the performance of the model.
3. Iteratively remove the worst performing features one by one till the overall performance of the model comes in acceptable range.

The performance metric used here to evaluate feature performance is pvalue. If the pvalue is **above 0.05** then we remove the feature, else we keep it.

Here we are using OLS model which stands for “Ordinary Least Squares”. This model is used for performing linear regression.

In [0]:
#Adding constant column of ones, mandatory for sm.OLS model
X_1 = sm.add_constant(X)
#Fitting sm.OLS model
model = sm.OLS(y,X_1).fit()
model.pvalues

In [0]:
# As we can see that the variable ‘AGE’ has highest pvalue of 0.9582293 which is greater than 0.05. 
# Hence we will remove this feature and build the model once again.

In [0]:
#Backward Elimination
# This is an iterative process and can be performed at once with the help of loop.
cols = list(X.columns)
pmax = 1
while (len(cols)>0):
    p= []
    X_1 = X[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print('Final set of variables:',selected_features_BE)

ii. RFE (Recursive Feature Elimination)
Works by recursively removing attributes and building a model on those attributes that remain.

Uses **accuracy metric** to rank the feature according to their importance.
1. Takes the model to be used and the number of required features as input.
2. Gives the ranking of all the variables, 1 being most important.

/here 7 (number of required features to remain) is rundom number. Better way is below.

In [0]:
model = LinearRegression()
#Initializing RFE model
rfe = RFE(model, 7)
#Transforming data using RFE
X_rfe = rfe.fit_transform(X,y)  
#Fitting the data to model
model.fit(X_rfe,y)
print(rfe.support_)
print(rfe.ranking_)

Here we took LinearRegression model with 7 features and RFE gave feature ranking as above, but the selection of **number ‘7’ was random**.

Now we need to find the *optimum* number of features.

We do that by using loop starting with 1 feature and going up to 13. We then take the one for which the accuracy is highest.

In [0]:
# № of features from 1 to original number of columns
nof_list=np.arange(1,13)            
high_score=0
#Variable to store the optimum features
nof=0           
score_list =[]
for n in range(len(nof_list)):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)
    model = LinearRegression()
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    model.fit(X_train_rfe,y_train)
    score = model.score(X_test_rfe,y_test)
    score_list.append(score)
    if(score>high_score):
        high_score = score
        nof = nof_list[n]
print("Optimum number of features: %d" %nof)
print("Score (R2) with %d features: %f" % (nof, high_score))

# the optimum number of features is 10

We now feed 10 as number of features to RFE and get the final set of features given by RFE method

In [0]:
cols = list(X.columns)
model = LinearRegression()
# Initializing RFE model
rfe = RFE(model, 10)             
# Transforming data using RFE
X_rfe = rfe.fit_transform(X,y)  
# Fitting the data to model
model.fit(X_rfe,y)              
temp = pd.Series(rfe.support_,index = cols)
selected_features_rfe = temp[temp==True].index
print(selected_features_rfe)

### Embedded Method:

Regularization methods are the most commonly used embedded methods which penalize a feature given a coefficient threshold.

If the feature is irrelevant, lasso penalizes it’s coefficient and make it 0. Hence the features with coefficient = 0 are removed and the rest are taken.

In [0]:
reg = LassoCV()
reg.fit(X, y)
print("Best alpha (penalizer) using built-in LassoCV: %f" % reg.alpha_)
print("Best score (R2) using built-in LassoCV: %f" %reg.score(X,y))

coef = pd.Series(reg.coef_, index = X.columns)
print("Lasso picked " + str(sum(coef != 0)) + 
      " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

In [0]:
imp_coef = coef.sort_values()
import matplotlib
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Feature importance using Lasso Model")
plt.xlabel('R2 score')
plt.ylabel('Features');

### Conclusion:

Following points will help you make the decision of which method to choose in what situation:
1. Filter method is less accurate. It is great while doing EDA, it can also be used for checking multi co-linearity in data.
2. Wrapper and Embedded methods give more accurate results but as they are computationally expensive, these method are suited when you have lesser features (~20).

## PCA

In [0]:
from sklearn.decomposition import PCA

ss = StandardScaler()
# Scale X_train.
X_train = ss.fit_transform(X_train)
# Scale X_test.
X_test = ss.transform(X_test)


pca = PCA(n_components=10)
# Instantiate linear regression model.
lm = LinearRegression()

# Transform Z_train and Z_test.
Z_train = pca.transform(X_train)
# Don't forget to transform the test data!
Z_test = pca.transform(X_test)

# Fit on Z_train.
lm.fit(Z_train, y_train)

# Score on training and testing sets.
print(f'Training Score: {round(lm.score(Z_train, y_train),4)}')
print(f'Testing Score: {round(lm.score(Z_test, y_test),4)}')

In [0]:
# Pull the explained variance attribute.
var_exp = pca.explained_variance_ratio_
print(f'Explained variance (first 20 components): {np.round(var_exp[:20],3)}')

print('')

# Generate the cumulative explained variance.
cum_var_exp = np.cumsum(var_exp)
print(f'Cumulative explained variance (first 20 components): {np.round(cum_var_exp[:20],3)}')